In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import numpy as np
%matplotlib inline
import re 
import math

In [2]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import spacy
# Need to load the large model to get the vectors
import en_core_web_sm
nlp = en_core_web_sm.load()

In [4]:
import os
import pandas as pd

# read the text file and add the column names
read_file = pd.read_csv(r"booksummaries.txt", sep='	', header=None)
read_file.columns = ['ID', 'm number', 'book name', 'author name', 'date', 'label', 'summary']

# clean data
read_file['label'] = read_file['label'].str.replace(r'/m/\S*\s', '')
read_file['label'] = read_file['label'].str.replace(r'{', '')
read_file['label'] = read_file['label'].str.replace(r'}', '')
read_file['label'] = read_file['label'].str.replace(r'}', '')
read_file['label'] = read_file['label'].str.replace(r'\\u00e0\s+clef', '')

# select columns
new_file = read_file.loc[:, ['book name', 'label', 'summary']]

#delete the columns with no labels
new_file.dropna(axis = 0, how = 'any', inplace = True)
new_file = new_file.iloc[:, [0, 2, 1]]

#output data as csv
new_file.to_csv(r'./booksummries.csv', index=False)


In [5]:
read_file.head()

,ID,m number,book name,author name,date,label,summary
0,620,/m/0hhy,Animal Farm,George Orwell,8/17/45,"""""Roman "", """"Satire"", """"Children's literature""...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"""""Science Fiction"", """"Novella"", """"Speculative ...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"""""Existentialism"", """"Fiction"", """"Absurdist fic...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"""""Hard science fiction"", """"Science Fiction"", ""...",The novel posits that space around the Milky ...


In [6]:
def cosine_similarity(a, b):
    return a.dot(b)/np.sqrt(a.dot(a) * b.dot(b))

In [7]:
def calculation(label_list):
    for i in range(len(label_list)):
        j = i+1
        if (j < len(label_list)):
            for j in range(len(label_list)):
                a = nlp(label_list[i]).vector
                b = nlp(label_list[j]).vector
                if cosine_similarity(a, b)>0.6:
                        if(len(label_list[i])>len(label_list[j])):
                            label_list[i] = label_list[j]
                        else:
                            label_list[j] = label_list[i]
    return set(label_list)

In [8]:
read_file["label"] = read_file["label"].fillna('')

In [9]:
for index in range(len(read_file['label'])):
    label = read_file['label'][index].replace('"', ''). lower()
    label_list = re.split(', ', label)  
    label_list = calculation(label_list)
    read_file.xs(index)['label']= label_list

In [10]:
read_file.head()

,ID,m number,book name,author name,date,label,summary
0,620,/m/0hhy,Animal Farm,George Orwell,8/17/45,"{roman , children's literature, fiction, satire}","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{novella, fiction, satire}","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,{novel},The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,{},The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,{fiction},The novel posits that space around the Milky ...
